In [76]:
import random

#returns a random Elliptic curve over a field GF(2^p)
#primeOrd forces returned EC to be of prime order

#general Weierstrass: Y^2 + a1*XY + a3*Y = X^3 + a2*X^2 + a4*X + a6
#here we will use curves E_(A, B)
#Y^2 + XY = X^3 + A*X^2 + B, A, B \in GF(2^p), 
#so coefs are [1, A, 0, 0, B]

def generateRandomEC(p, primeOrd=False, maxAttempts = 256):
    T = GF(2**p)
    attempts = 0
    maxFactor = 0
    bestCoefs = []
    while True:
        coefs = [1, -1, 0, 0, -1]
        coefs[1] = T.random_element() #A
        coefs[4] = T.random_element() #B - shouldn't be 0
        
        try:
            E = EllipticCurve(T, coefs)
            if primeOrd == False or is_prime(E.order()) or attempts > maxAttempts:
                break
            else:
                if max(prime_factors(E.order())) > maxFactor: # store highest factor
                    maxFactor = max(prime_factors(E.order()))
                    bestCoefs = coefs
                attempts += 1 #to avoid infinite loop if there isnt a prime-order EC
        except ArithmeticError: #if E singular
            pass
    if primeOrd and is_prime(E.order()) == False:
        E = EllipticCurve(T, bestCoefs)
        print("Prime order EC requested, after {} attempts, EC is prime: {}.\n".format(attempts, is_prime(E.order())))
    print(E)
    return E



def generateRandomECPrime(p, primeOrd=False, maxAttempts = 256):
    T = GF(next_prime(2**p))
    attempts = 0
    maxFactor = 0
    bestCoefs = []
    while True:
        coefs = [1, -1, 0, 0, -1]
        coefs[1] = T.random_element() #A
        coefs[4] = T.random_element() #B - shouldn't be 0
        
        try:
            E = EllipticCurve(T, coefs)
            if primeOrd == False or is_prime(E.order()) or attempts > maxAttempts:
                break
            else:
                if max(prime_factors(E.order())) > maxFactor: # store highest factor
                    maxFactor = max(prime_factors(E.order()))
                    bestCoefs = coefs
                attempts += 1 #to avoid infinite loop if there isnt a prime-order EC
        except ArithmeticError: #if E singular
            pass
    if primeOrd and is_prime(E.order()) == False:
        E = EllipticCurve(T, bestCoefs)
        print("Prime order EC requested, after {} attempts, EC is prime: {}.\n".format(attempts, is_prime(E.order())))
    print(E)
    return E


#returns 3rd summation polynomial for a curve E_(A, B) = Y^2 + XY = X^3 + A*X^2 + B, A, B \in GF(2^p), 
def genSM3(FF, B):
    PR.<x1, x2, x3> = PolynomialRing(FF, order='degrevlex')
    return ((x1*x2 + x1*x3 + x2*x3)^2 + x1*x2*x3 + B) 

#returns a random multiple of a base point P and order of subgroup <P>
def generateRandomPoint(P):
    orderP=P.order()
    k=int(1+(orderP-1)*random.random()) #k in [1, orderP-1]
    Q=k*P
    return Q, orderP

In [2]:
#Individual step of Pohlig-Hellman algorithm
#Solves DLP in a subgroup of order 'orderP'
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Pi - generator of the subgroup, Qi = ki*Pi, order of Pi is orderP, returns ki
#Splits ki = x0 + p*x1 + p^2*x2 + p^(e-1)*x_(e-1) and solves 'e'-times DLP in a subgroup of order 'p'
def PH_reduction(Qi, Pi, orderP, **kwargs): 
    invPi = -Pi #additive inverse
    p = prime_factors(orderP)[0]
    e = orderP.valuation(p)
    
    Pi = (p^(e-1))*Pi
    ki = 0 #ki = x0 + p*x1 + p^2*x2 + ...
    for i in srange(0, e):
        tempQ = (p^(e-1-i))*(Qi + ki*invPi)
        ki += (p^i)*discrete_log_rho(tempQ, Pi, operation="+")
    return ki


#Implements Pohlig-Hellman algorithm
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Solves Q = k*P, returns k
#P generates a subgroup of order orderP
#DLP_Solve is a function solving ECDLP in a subgroup of order p^e, where 'p' is a prime number
def Pohlig_Hellman_additive(Q, P, orderP, DLP_Solve = PH_reduction, **kwargs):
    print("Factorization of orderP: {} = {}.".format(orderP, orderP.factor()))
    print("Order of E = <P> is {}.\nBit security (log2 of the biggest prime_factor of orderP): {:4.1f}"\
      .format(orderP, float(log(max(prime_factors(orderP)))/log2)))
    
    ki = [] #list of individual DLP results
    mi = [] #list of moduli
    for p in prime_factors(orderP): #prime factor p
        e = orderP.valuation(p) #multiplicity of p in orderP
        mi.append(p^e) 
        pei = Integer(orderP/mi[-1])
        ki.append(DLP_Solve(pei*Q, pei*P, mi[-1], **kwargs)) #solves ECDLP in a subgroup <pei*P>
    return Integer(CRT(ki, mi))


In [64]:
import random
#Generates a base of random distinct points (a*P + b*Q), size depends on 'm'
#Based on (Algorithm 4.1) https://eprint.iacr.org/2017/1262.pdf page 10
#Input: P - base point
#       Q = k*P
#       orderP is order of group <P>
#Returns:   coord, factorBase, res
#           coord is a list of (a_i, b_i) coefficients
#           factorBase is a list of factorBase points (a_i*P + b_i*Q)
#           If res != -1 it's a solution to the ECDLP => res*P = Q
def buildFactorBase(Q, P, orderP, m = 3):
    factorBaseSize = ceil(orderP^(1/m)) 
    factorBase = []
    coord = []
    currentSize = 0
    res = -1
    while currentSize < factorBaseSize:
        a = int(random.random()*orderP) #a,b in [0, orderE - 1]
        b = int(random.random()*orderP)
        candidate=a*P+b*Q
        if candidate not in factorBase:
            factorBase.append(candidate)
            coord.append((a,b))
            currentSize += 1
        else: #we can try to solve the whole problem here
            c, d = coord[factorBase.index(candidate)]
            
            #relationship: a*P + b*Q = c*P + d*Q => (a-c)*(d-b)^(-1)*P=Q
            try:
                res = Integer(mod((a - c)*inverse_mod(d - b, orderP), orderP))
                break
            except ZeroDivisionError: #inverse of 'b' mod 'orderP' does not exist
                pass
                
    return coord, factorBase, res


#Multi-index class, dimension is dim
#[i_1, i_2, ..., i_dim]
class MultiIndex:
    def __init__(self, dim, baseSize):
        self.dim = dim
        self.baseSize = baseSize
        self.index = [0 for i in srange(0, self.dim)]
        
    def dbgPrint(self):
        print("Dim: {}, baseSize: {}, index: {}.".format(self.dim, self.baseSize, self.index))

    def getIndex(self):
        return self.index
    
    #returns a multi-index, -1 means the index is out of bounds
    def nextIndex(self):
        isDone = True
        for i in srange(self.dim - 1, -1, -1):
            if (self.index[i] + 1) < self.baseSize:
                isDone = False
                break
        if isDone == True:
            return [-1]
        
        self.index[i] += 1
        for k in srange(i + 1, self.dim):
            self.index[k] = self.index[i]
        return self.index

In [89]:
import time

def sumInF(Q, P, orderP, **kwargs):
    m = 3
    maxAttempts = 16
    for key, value in kwargs.items():
        if key == "m":
            m = Integer(value)
        elif key == "maxAttempts":
            maxAttempts = Integer(value)

    for attempt in srange(0, maxAttempts):
        coord, factorBase, res = buildFactorBase(Q, P, orderP, m)
        if res != -1:
            return res

        length = len(factorBase)
        print("Attempt: {}, EC order: {}, factor base size: {}.".format(attempt+1, orderP, length))
        dim = m - 1
        multiIndex = MultiIndex(dim, length)
        index = multiIndex.getIndex()

        while index[0] != -1:
            dp = [factorBase[i] for i in index]
            for v in VectorSpace(GF(2), dim):
                test = sum(-dp[k] if v[k] else dp[k] for k in srange(0, dim))

                if test in factorBase:
                    baseId = factorBase.index(test)

                    b = coord[baseId][1] + sum(coord[index[k]][1] if v[k] 
                                              else -coord[index[k]][1] for k in srange(0,dim))
                    a = -coord[baseId][0] + sum(-coord[index[k]][0] if v[k] 
                                              else coord[index[k]][0] for k in srange(0,dim))
                    try:
                        return Integer(mod(a*inverse_mod(b, orderP), orderP))
                    except ZeroDivisionError: #inverse of 'b' mod 'orderP' does not exist
                        pass
            index = multiIndex.nextIndex()
    return -1

def bruteForce(P, Q, orderP):
    for i in srange(0, orderP):
        if (i*P == Q):
            return i
    raise ValueError #no solution to this DLP

def sum2inF(Q, P, orderP, **kwargs):
    maxAttempts = 16
    for key, value in kwargs.items():
        if key == "maxAttempts":
            maxAttempts = value
            break
            
    for attempt in srange(0, maxAttempts):
        coord, factorBase, res = buildFactorBase(Q, P, orderP)
        if res != -1:
            return res
        
        length = len(factorBase)
        print("Attempt: {}, EC order: {}, factor base size: {}.".format(attempt+1, orderP, length))
        for i in srange(0, length):
            for j in srange(i, length):
                ids = [i, j]
                for v in VectorSpace(GF(2), 2):
                    test = sum(-factorBase[ids[k]] if v[k] else factorBase[ids[k]] for k in [0,1]);

                    if test in factorBase:
                        baseId = factorBase.index(test)

                        b = coord[baseId][1] + sum(coord[ids[k]][1] if v[k] 
                                                   else -coord[ids[k]][1] for k in [0,1])

                        a = -coord[baseId][0] + sum(-coord[ids[k]][0] if v[k] 
                                                    else coord[ids[k]][0] for k in [0,1])

                        try:
                            res = Integer(mod(a*inverse_mod(b, orderP), orderP))
                            return res
                        except ZeroDivisionError: #inverse of 'b' mod 'orderP' does not exist
                            pass
    raise ValueError #if logarithm wasnt found

#import timeit
#print(timeit.timeit('sum2inF(coord, factorBase, orderP)', 'from __main__ import sum2inF, coord, factorBase, orderP', number=100))
#print(timeit.timeit('sumInF(coord, factorBase, orderP, 3)', 'from __main__ import sumInF, coord, factorBase, orderP', number=100))
#prime

#primes
#11 | 13 | 17 | 19 | 23 | 29 | 31 | 37 | 41 | 43 | 47 | 53 | 59 | 61 | 67 | 71 | 73 | 79 | 83 | 89 | 97

# p = 31 - 963 s Sum2F 1 attempt
# p = 37 - too long

p = 19
E = generateRandomECPrime(p, True)
#E = generateRandomEC(p, True)
P = E.gen(0) #base point P
Q, orderP = generateRandomPoint(P)
print("Order of P is {}.".format(orderP))

print("Factorization of Elliptic curve order: {} = {}".format(orderP, orderP.factor()))

tm = time.time()
r1 = discrete_log(Q,P, E.order(), operation='+')
print("Sage: It took {:4.4f} seconds. Result isValid: {}.\n"\
       .format((time.time()-tm), bool(r1*P == Q)))

tm = time.time()
r1 = Pohlig_Hellman_additive(Q,P, E.order())
print("PH-rho: It took {:4.4f} seconds. Result isValid: {}.\n"\
       .format((time.time()-tm), bool(r1*P == Q)))

# start = time.time()
# res = bruteForce(P, Q, orderP)
# print("It took {:4.4f} seconds. Brute force. ".format(time.time()-start))
      
tm = time.time()
while True:
    res = Pohlig_Hellman_additive(Q, P, orderP, sum2inF)

    if res != -1:
        break
tm = time.time() - tm
print("Sum2F: It took {:4.4f} seconds. Result isValid: {}.\n"\
      .format(tm, bool(res*P == Q)))
assert(res*P == Q)

for m in srange(3, 7):
    tm = time.time()
    kwargs = {"m" : m, "maxAttempts" : 25}
    while True:
        res=Pohlig_Hellman_additive(Q, P, orderP, sumInF, **kwargs)
        if res != -1 or attempts > 20:
            break
    tm = time.time() - tm
    print("SumInF - for m = {} it took {:4.4f} seconds. Result isValid: {}.\n"\
          .format(m, tm, bool(res*P == Q)))

Elliptic Curve defined by y^2 + x*y = x^3 + 179769*x^2 + 395240 over Finite Field of size 524309
Order of P is 523657.
Factorization of Elliptic curve order: 523657 = 523657
Sage: It took 0.1127 seconds. Result isValid: True.

Factorization of orderP: 523657 = 523657.
Order of E = <P> is 523657.
Bit security (log2 of the biggest prime_factor of orderP): 19.0
PH-rho: It took 0.4021 seconds. Result isValid: True.

Factorization of orderP: 523657 = 523657.
Order of E = <P> is 523657.
Bit security (log2 of the biggest prime_factor of orderP): 19.0
Attempt: 1, EC order: 523657, factor base size: 81.


KeyboardInterrupt: 

In [5]:
# class MultiIndex:
#     def __init__(self, m, orderP):
#         self.dim = m - 1
#         self.orderP = orderP
#         self.index = [0 for i in range(0, self.dim)]
        
#     def dbgPrint(self):
#         print("Dim: {}, order: {}, index: {}.".format(self.dim, self.orderP, self.index))
        
#     #returns a multi-index, -1 means the index is out of bounds
#     def nextIndex(self):
#         isDone = True
#         for i in range(self.dim - 1, -1, -1):
#             if (self.index[i] + 1) < self.orderP:
#                 isDone = False
#                 break
#         if isDone == True:
#             return [-1]
        
#         self.index[i] += 1
#         for k in range(i + 1, self.dim):
#             self.index[k] = self.index[i]
#         return self.index

In [17]:
import random
#Generates random distinct points (a*P + b*Q), size depends on 'm'
#Stores only x-coordinate
#Based on (Algorithm 2.5) https://eprint.iacr.org/2017/1262.pdf page 10
#Input: P - base point
#       Q = k*P
#       orderP is order of group <P>
#Returns:   coord, factorBase, res
#           coord is a list of (a_i, b_i) coefficients
#           factorBase is a list of x_i of points (a_i*P + b_i*Q)
#           If res != -1 it's a solution to the ECDLP => res*P = Q
# F -- Finite Field
def buildFactorBaseOnlyX(Q, P, orderP, FF, m = 3):
    factorBaseSize = ceil(orderP^(1/m)) 
    factorBase = []
    
    #index used in factorBasePoly set
    k = IntegerModRing(m)(0) 
    #polynomial ring in 'X'
    PR.<X> = PolynomialRing(FF, order='degrevlex')

    #sets partitioning, init to 1 (product)
    #contains f_i(X) polynomials
    factorBasePoly = [ 1 for i in range(0, m)] 
    
    coord = []
    currentSize = 0
    while currentSize < factorBaseSize:
        a = int(random.random()*orderP) #a,b in [0, orderE - 1]
        b = int(random.random()*orderP)
        candidate=(a*P+b*Q)[0] #we are only interested in the x-coordinate
        if candidate not in factorBase:
            factorBase.append(candidate)
            coord.append((a,b))
            currentSize += 1
            
            #(X - v), v \in V_i
            #generally (X - candidate)
            #bc field char is 2
            factorBasePoly[k] *= (X - candidate) 
            k += 1
        #else: TODO - can we solve ECDLP directly here?
        
    return coord, factorBase, factorBasePoly

def findPoints(xs, E):
    relation = []
    points = []
    ident = E(0) #infinity point (identity)
    xs = [xs["x1"], xs["x2"], xs["x3"]]
    for val in xs:
        if E.is_x_coord(val): #valid point
            adept = E.lift_x(val, all=False, extend=False)
            if adept != ident: #no new information
                points.append(adept)
    #find signs
    dim = len(points)
    if dim > 1:
        for v in VectorSpace(GF(2), dim - 1):
            if (points[-1] + sum(-points[k] if v[k] else points[k] for k in range(0, dim - 1))) == ident:
                relation = [-points[k] if v[k] else points[k] for k in range(0, dim - 1)]
                relation.append(points[-1])
                break
    return relation
    

In [62]:
import time
#input: E - Elliptic curve - over GF(2^p)
#       P - base point P
#       Q - k*P
#       FF - Finite Field - GF(2^p)
#       orderP - order of group <P>
#       coord is a list of (a_i, b_i) coefficients
#       factorBase is a list of x_i of points (a_i*P + b_i*Q)
#       factorBasePoly is [ f_i(X) i in range(0, m) ]
#       where f_i(X) is product(X - v_i), v_i \in V_i (set)

def alg2_5(E, P, Q, FF, orderP, coord, factorBase, factorBasePoly, m = 3):
    PR.<x1, x2, x3> = PolynomialRing(FF,3, order='degrevlex')
    SM3 = genSM3(FF, E.a6()) #3rd summation polynomial 
    res = -1
    
    generators = [SM3(x1,x2,x3)] #add summation polynomial
    #f_1(x_1)
    generators.append(factorBasePoly[0](x1))
    #f_2(x_2)
    generators.append(factorBasePoly[1](x2))
    #f_3(x_3)
    generators.append(factorBasePoly[2](x3))
    
    #calculate Groebner basis of an ideal <generators>
    tm=time.time()
    tmp = PR.ideal(generators);
    gb = tmp.groebner_basis('libsingular:groebner')
    print("GB generation took: {} s.".format(time.time() - tm))
    sys.stdout.flush()
    #solutions to the polynomial system
    variety = (PR.ideal(gb)).variety() 

 #   print("Variety-Solution of len {}, GB: {}.".format(len(variety), gb))
  
    #array of dicts [ {"x1": a,"x2": b,"x3": c}]
    #E(0) - identity element
    for solution in variety:
        relation = findPoints(solution, E)
        sumA = 0
        sumB = 0
        for rel in relation:
            if rel[0] in factorBase:
                baseId = factorBase.index(rel[0])
                if ( (coord[baseId][0]*P + coord[baseId][1]*Q) == rel ):
                    sumA -= coord[baseId][0]
                    sumB += coord[baseId][1]
                else: # (-rel) is in factorBase
                    sumA += coord[baseId][0]
                    sumB -= coord[baseId][1]
            else:
                print("something fishy...")
        try:
    #        print("SumA: {}, SumB: {}, orderP: {}.".format(sumA, sumB, orderP))
            res = Integer(mod(sumA*inverse_mod(sumB, orderP), orderP))
            break
        except ZeroDivisionError: #inverse of 'sumB' mod 'orderP' does not exist
            print("SumA = {}, SumB  = {} not invertible mod orderP = {}.".format(sumA, sumB, orderP))
            pass

    return res


#### Tests:
p = 17
E = generateRandomEC(p, primeOrd=True)
P = E.gen(0) #base point P
Q, orderP = generateRandomPoint(P)
FF = GF(2^p)
print("Order of P is {}.".format(orderP))

print("Factorization of Elliptic curve order: {} = {}".format(orderP, orderP.factor()))
sys.stdout.flush()

#print(coord)
#print(factorBase)
#print(factorBasePoly)
good = 0
totalTime = time.time()
attempts = 0
while True:
    coord, factorBase, factorBasePoly = buildFactorBaseOnlyX(Q, P, orderP, FF)
    print("Factorization base of lenght {} built.".format(len(factorBase)))
    sys.stdout.flush()
        
    res = alg2_5(E, P, Q, FF, orderP, coord, factorBase, factorBasePoly, m = 3)
    print("Attempt {}, result: {}, res: {}.\n\n".format(attempts, bool(res*P == Q), res))
    sys.stdout.flush()
    
    good += int(res*P == Q)
    attempts += 1
    if good:
        break
print("Attempt success rate: {}/{}, totalTime: {:4.4f} seconds.".format(good, attempts, time.time()-totalTime))

#tm = time.time()
#r1 = discrete_log(Q,P, E.order(), operation='+')
#print("Sage: It took {:4.4f} seconds. Result isValid: {}.\n"\
 #      .format((time.time()-tm), bool(r1*P == Q)))


    

Prime order EC requested, after 257 attempts, EC is prime: False.

Elliptic Curve defined by y^2 + x*y = x^3 + (z17^16+z17^13+z17^12+z17^10+z17^9+z17^8+z17^6+z17^4+z17^3+z17^2+1)*x^2 + (z17^16+z17^15+z17^9+z17^7+z17^4+z17^3+z17^2+z17+1) over Finite Field in z17 of size 2^17
Order of P is 131414.
Factorization of Elliptic curve order: 131414 = 2 * 65707
Factorization base of lenght 51 built.
GB generation took: 440.944166183 s.
SumA = 193850, SumB  = -230224 not invertible mod orderP = 131414.
Attempt 0, result: False, res: -1.


Factorization base of lenght 51 built.
GB generation took: 463.342492819 s.
Attempt 1, result: False, res: -1.


Factorization base of lenght 51 built.
GB generation took: 475.760105133 s.
Attempt 2, result: False, res: -1.


Factorization base of lenght 51 built.
GB generation took: 457.313647985 s.
Attempt 3, result: False, res: -1.


Factorization base of lenght 51 built.


KeyboardInterrupt: 

In [10]:
#VF = VectorSpace(RR, 3)
#test commands
P.<x,y> = PolynomialRing(GF(29))
print(P)
I = Ideal([2*x^2+3*y^2-11, x^2-y^2-3, x*y + x^2 -2])
gb = I.groebner_basis()
#V(2x2 + 3y2 − 11, x2 − y2 − 3)
igb = Ideal(gb)
iv = igb.variety()
print(iv)
for rel in iv:
    ix = rel["x"]
    iy = rel["y"]


Multivariate Polynomial Ring in x, y over Finite Field of size 29
[{y: 1, x: 27}, {y: 28, x: 2}]
(27, 1)
(2, 28)


In [12]:
# def genSM3(FF):
#     PR.<x1, x2, x3> = PolynomialRing(FF, order='degrevlex')
#     return ((x1*x2 + x1*x3 + x2*x3)^2 + x1*x2*x3 + 1) #TODO: generalize to different ECs
# FF = GF(2^5)
# PR2.<X> = PolynomialRing(FF, "X", 1, order='degrevlex')

# PR.<x1, x2, x3> = PolynomialRing(FF, order='degrevlex')
# SM3 = genSM3(FF) 
# o = (X + FF.random_element())
# o.groebner_basis()
# print(o(x1), o)

In [36]:
FF.<a> = GF(2^3)
d1 = FF.random_element()
d2 = d1
coefs = [1, 1, 0, 0, 1]
E = EllipticCurve(FF, coefs)
k = E.random_element()
print(k, k + E(0))
print(E)
import sys

((a^2 + a : a + 1 : 1), (a^2 + a : a + 1 : 1))
Elliptic Curve defined by y^2 + x*y = x^3 + x^2 + 1 over Finite Field in a of size 2^3
(2, 7)


In [18]:
# toweierstrass u = d1(d1^2+d1+d2)(x+y)/(x y+d1(x+y))
# toweierstrass v = d1(d1^2+d1+d2)(x/(x y+d1(x+y))+d1+1)
#fromweierstrass x = d1(u+d1^2+d1+d2)/(u+v+(d1^2+d1)(d1^2+d1+d2))
#fromweierstrass y = d1(u+d1^2+d1+d2)/(v+(d1^2+d1)(d1^2+d1+d2))
# a1 = 1
# a2 = d1^2+d2
# a3 = 0
# a4 = 0
# a6 = d1^4(d1^4+d1^2+d2^2)

FF = GF(2^19)
d1 = FF.random_element()
d2 = d1
coefs = [1, d1^2 + d2, 0, 0, d1^4*(d1^4+d1^2+d2^2)]
E = EllipticCurve(FF, coefs)


#u = d1(d1^2+d1+d2)(x+y)/(x y+d1(x+y))
print(E)
Er=E.random_element()
u=Er[0]
v=Er[1]
x = d1*(u+d1^2+d1+d2)/(u+v+(d1^2+d1)*(d1^2+d1+d2))
y = d1*(u+d1^2+d1+d2)/(v+(d1^2+d1)*(d1^2+d1+d2))
print(x,y)
print(d1*(x+y)+d2*(x^2+y^2)-x*y-x*y*(x+y)-x^2*y^2)

In [16]:
FF=GF(2^7)
PR = PolynomialRing(FF, "x", 3, order='degrevlex')
print(PR)
var('x1,x2,x3,x4,y1,y2,y3,a4,a6,X')
lambda3=(y2-y1)/(x2-x1)
lambda4=(-x2-y2-y1)/(x2-x1)

#EC: Y^2 + a1*XY + a3*Y = X^3 + a2*X^2 + a4*X + a6
#negation -P = (x1, -a1*x1 - a3 - y1)

#for P+Q = (x1, y1) + (x2, y2), x1 != x2
#addition slope lambda = (y2 - y1)/(x2 - x1) #binary: (y2 - y1) = (y2 + y1)

# (x3, y3) = (x1, y1) + (x2, y2)
# x3 = lambda^2 + a1*lambda -a2 -x1 -x2

#3rd summ poly: Choose points (x1, y1), (x2, y2) so x1 != x2
# (x3, y3) = (x1, y1) + (x2, y2) 
# (x4, y4) = (x1, y1) - (x2, y2)
# express x3 and x4, use field char to simplify
# express: x3 + x4 and x3*x4, use EC definition to sub. for y^2
# x3, x4 are roots of the polynomial => build the quadratic polynomial
# x3 + x4 = -b/a, x3*x4 = c/a

x3=lambda3^2+lambda3-x1-x2
x4=lambda4^2+lambda4-x1-x2
res=x3+x4
#print(latex(res.full_simplify())) #mod 2

res=x3*x4
a=x1^2*x2+x1*x2^2-a4*x1^2-x1*x2-x1*y2+x2*y1-a4*x2^2
b=x1^2*x2+x1*x2^2-a4*(x1^2+x2^2)
latex((a*b).full_simplify()) #mod 2
#print(res) #go to wolfram mathematica
E = generateRandomEC(3)
print(E.a4())

Multivariate Polynomial Ring in x0, x1, x2 over Finite Field in z7 of size 2^7
Elliptic Curve defined by y^2 + x*y = x^3 + (z3+1)*x^2 + (z3^2+1) over Finite Field in z3 of size 2^3
0
